In [1]:
import requests
import os
import pandas as pd
import datetime

In [2]:
bearer_token = "BQCO5DgRhiwYHB9zy8wLuVlc4XKXYrvbfz_7pEzXDMOF6Lb7ThzwoEm1O9YSnhyC_VmMKa-0NmYetjzq0FwOtpb99SfWvLO2DJWPiQHPdfEXaraLKgoyJ1vcSHRjKv0tzDPPvhA1V2OnDoiqhbs0A_b0pV9wdBOZHehfQ8krFUcMQUAoMpq5jM_QCwbogY-DhMF4TOAIX0eTRFIPJiYGH4IA-sTZyPrnjTxzwenX7p99oQz8Yb8C52FvFTNAMrT7"

In [3]:
endpoint = 'https://charts-spotify-com-service.spotify.com/auth/v0/charts/artist-{country}-weekly/{date}'

In [4]:
database_dir = 'data'
os.makedirs(database_dir, exist_ok=True)

In [5]:
headers = {
    'Authorization': f'Bearer {bearer_token}',
}

In [6]:
countries = {
    "GLOBAL": "Global",
    "AR": "Argentina",
    "AU": "Australia",
    "AT": "Austria",
    "BY": "Belarus",
    "BE": "Belgium",
    "BO": "Bolivia",
    "BR": "Brazil",
    "BG": "Bulgaria",
    "CA": "Canada",
    "CL": "Chile",
    "CO": "Colombia",
    "CR": "Costa Rica",
    "CY": "Cyprus",
    "CZ": "Czech Republic",
    "DK": "Denmark",
    "DO": "Dominican Republic",
    "EC": "Ecuador",
    "EG": "Egypt",
    "SV": "El Salvador",
    "EE": "Estonia",
    "FI": "Finland",
    "FR": "France",
    "DE": "Germany",
    "GR": "Greece",
    "GT": "Guatemala",
    "HN": "Honduras",
    "HK": "Hong Kong",
    "HU": "Hungary",
    "IS": "Iceland",
    "IN": "India",
    "ID": "Indonesia",
    "IE": "Ireland",
    "IL": "Israel",
    "IT": "Italy",
    "JP": "Japan",
    "KZ": "Kazakhstan",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "MY": "Malaysia",
    "MX": "Mexico",
    "MA": "Morocco",
    "NL": "Netherlands",
    "NZ": "New Zealand",
    "NI": "Nicaragua",
    "NG": "Nigeria",
    "NO": "Norway",
    "PK": "Pakistan",
    "PA": "Panama",
    "PY": "Paraguay",
    "PE": "Peru",
    "PH": "Philippines",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SA": "Saudi Arabia",
    "SG": "Singapore",
    "SK": "Slovakia",
    "ZA": "South Africa",
    "KR": "South Korea",
    "ES": "Spain",
    "SE": "Sweden",
    "CH": "Switzerland",
    "TW": "Taiwan",
    "TH": "Thailand",
    "TR": "Turkey",
    "AE": "UAE",
    "UA": "Ukraine",
    "GB": "United Kingdom",
    "UY": "Uruguay",
    "US": "USA",
    "VE": "Venezuela",
    "VN": "Vietnam"
}

In [7]:
all_counties = list(countries.keys())

In [8]:
def get_weekly_song_rankings(start_date, country_code):
    response = requests.get(endpoint.format(country=country_code, date=start_date), headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

In [9]:
date_format = '%Y-%m-%d'

In [10]:
def weekly_to_lines(weekly_json):
    rankings = []
    for entry in weekly_json['entries']:
        ranking = dict(entry['chartEntryData'])
        ranking['artist_id'] = entry['artistMetadata']['artistUri'].split(':')[-1]
        ranking['country'] = weekly_json['displayChart']['chartMetadata']['dimensions']['country']
        ranking['date'] = weekly_json['displayChart']['date']
        ranking['chartType'] = weekly_json['displayChart']['chartMetadata']['dimensions']['chartType']  
        ranking['artist_name'] = entry['artistMetadata']['artistName']
        ranking['image_link'] = entry['artistMetadata']['displayImageUri']      
        rankings.append(ranking.copy())

    return rankings

In [ ]:
start_date = datetime.datetime(2025, 5, 15)

# Create empty pandas DataFrames to store the rankings and artists
rankings_db = []

for country_code in all_counties:
    json_data = get_weekly_song_rankings('latest', country_code.lower())
    earliest_date = json_data['displayChart']['chartMetadata']['dimensions']['earliestDate']
    date = start_date

    while date.strftime(date_format) >= earliest_date:
        date_str = date.strftime(date_format)
        country_lower = country_code.lower()
        json_data = get_weekly_song_rankings(date_str, country_lower)
        if json_data is None:
            print(f"Error at {date_str}, {country_code} due to error.")
            continue

        ranking = weekly_to_lines(json_data)
        rankings_db.extend(ranking)

        # Move date to previous week
        date -= datetime.timedelta(weeks=1)

Error: 504
Error at 2024-02-29, CA due to error.
Error: 504
Error at 2024-11-14, EE due to error.
Error: 504
Error at 2025-03-06, DE due to error.
Error: 504
Error at 2025-01-09, HK due to error.
Error: 504
Error at 2023-11-16, MY due to error.
Error: 504
Error at 2025-05-08, NG due to error.
Error: 504
Error at 2023-03-23, NG due to error.
Error: 504
Error at 2025-02-27, PY due to error.
Error: 504
Error at 2024-11-21, PY due to error.
Error: 504
Error at 2024-06-13, PL due to error.
Error: 504
Error at 2024-02-08, SG due to error.


In [17]:
# Save databases
rankings_df = pd.DataFrame(rankings_db, columns=['artist_id', 'country', 'date', 'chartType', 'artist_name', 'image_link', "currentRank", "previousRank", "peakRank", "peakDate", "appearancesOnChart", "consecutiveAppearancesOnChart", "entryStatus", "entryRank", "entryDate"])
rankings_df.to_csv(os.path.join(database_dir, 'rankings.csv'), index=False)

In [18]:
len(rankings_df)

2747200

In [19]:
rankings_df.keys()

Index(['artist_id', 'country', 'date', 'chartType', 'artist_name',
       'image_link', 'currentRank', 'previousRank', 'peakRank', 'peakDate',
       'appearancesOnChart', 'consecutiveAppearancesOnChart', 'entryStatus',
       'entryRank', 'entryDate'],
      dtype='object')

In [21]:
rankings_df.head(10)

,artist_id,country,date,chartType,artist_name,image_link,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,entryStatus,entryRank,entryDate
0,4q3ewBCX7sLwd24euuV69X,GLOBAL,2025-05-15,TOP_ARTIST,Bad Bunny,https://i.scdn.co/image/ab6761610000517481f47f...,1,1,1,2021-10-28,187,187,NO_CHANGE,2,2021-10-21
1,06HL4z0CvFAxyc27GXpf02,GLOBAL,2025-05-15,TOP_ARTIST,Taylor Swift,https://i.scdn.co/image/ab67616100005174e672b5...,2,2,1,2021-11-11,187,187,NO_CHANGE,6,2021-10-21
2,1Xyo4u8uXC1ZmMpatF05PJ,GLOBAL,2025-05-15,TOP_ARTIST,The Weeknd,https://i.scdn.co/image/ab676161000051749e5289...,3,4,1,2022-01-13,187,187,MOVED_UP,11,2021-10-21
3,3TVXtAsR1Inumwj472S9r4,GLOBAL,2025-05-15,TOP_ARTIST,Drake,https://i.scdn.co/image/ab67616100005174429338...,4,3,1,2023-10-12,187,187,MOVED_DOWN,3,2021-10-21
4,0ys2OFYzWYB5hRDLCsBqxt,GLOBAL,2025-05-15,TOP_ARTIST,Fuerza Regida,https://i.scdn.co/image/ab67616100005174158a3d...,5,7,5,2025-05-15,128,128,MOVED_UP,190,2022-12-08
5,6qqNVTkY8uBg9cP3Jd7DAH,GLOBAL,2025-05-15,TOP_ARTIST,Billie Eilish,https://i.scdn.co/image/ab676161000051744a21b4...,6,6,2,2024-05-23,187,187,NO_CHANGE,12,2021-10-21
6,1HY2Jd0NmPuamShAr6KMms,GLOBAL,2025-05-15,TOP_ARTIST,Lady Gaga,https://i.scdn.co/image/ab67616100005174aadc18...,7,5,4,2025-03-13,187,187,MOVED_DOWN,55,2021-10-21
7,2YZyLoL8N0Wb9xBt1NhZWg,GLOBAL,2025-05-15,TOP_ARTIST,Kendrick Lamar,https://i.scdn.co/image/ab6761610000517439ba6d...,8,8,1,2024-11-28,187,187,NO_CHANGE,53,2021-10-21
8,66CXWjxzNUsdJxJ2JdwvnR,GLOBAL,2025-05-15,TOP_ARTIST,Ariana Grande,https://i.scdn.co/image/ab67616100005174672580...,9,9,2,2024-03-14,187,187,NO_CHANGE,17,2021-10-21
9,4YRxDV8wJFPHPTeXepOstw,GLOBAL,2025-05-15,TOP_ARTIST,Arijit Singh,https://i.scdn.co/image/ab676161000051745ba2d7...,10,11,7,2023-11-09,187,187,MOVED_UP,100,2021-10-21
